# Data Capture

In [1]:
with open("https://raw.githubusercontent.com/z3r0st/TF-201711448-20181C074-202021767/main/SF_street_intersections.csv") as f:
    data = f.readlines()[1:]
    intercepts = dict()
    streets = dict()
    nodes = dict()
    nodeToIntercept = dict()
    c = 0
    for line in data:
        street, point = line.strip().split(sep=',')[1:3]
        lon, lat = point[7:-2].split()
        intercept = (float(lat), float(lon))
        if street in intercepts.keys():
            intercepts[street].append(intercept)
        else:
            intercepts[street] = [intercept]
        if intercept in streets.keys():
            streets[intercept].append(street)
        else:
            streets[intercept] = [street]
            nodes[intercept] = c
            nodeToIntercept[c] = intercept
            c += 1

In [2]:
intercepts['UTAH']

[(37.7543372313743, -122.40545417189192),
 (37.76841488233248, -122.40679858931149),
 (37.7607529052339, -122.40606109793055),
 (37.7619964030768, -122.4061819835959),
 (37.7658422711544, -122.40655282446389),
 (37.7512810744602, -122.40515881630326),
 (37.7594458071505, -122.40593403378597),
 (37.769298557428, -122.40688239974939),
 (37.7562225803215, -122.40521698628322),
 (37.7645476536961, -122.40642915445186),
 (37.7517869480963, -122.40520770296072),
 (37.7632538870406, -122.40630557076001),
 (37.75692043860479, -122.40525954218414),
 (37.753063831974, -122.40533110370748),
 (37.7671421818598, -122.40667700476389)]

In [4]:
streets[(37.7543372313743, -122.40545417189192)]

['UTAH', '23RD']

# Graph creation

In [5]:
import math

In [6]:
def calculateDistance(p1, p2):
    return math.sqrt(sum((map(lambda c1, c2: (abs(c1 - c2))**2, p1, p2))))

In [7]:
def createGraph(streets, intercepts, nodes):
    G = [[] for i in range(len(streets.keys()))]
    
    for intercept in streets.keys():
        for street in streets[intercept]:
            neighbours = []
            for point in intercepts[street]:
                if point != intercept:
                    neighbours.append([point, calculateDistance(intercept, point)])
            
            if len(neighbours) > 1:
                neighbours.sort(key=lambda ls: ls[1])
                neighbours = neighbours[0:2]
                if calculateDistance(neighbours[0][0], neighbours[1][0]) < neighbours[0][1]:
                    neighbours.pop()
                
                G[nodes[intercept]].extend([(nodes[neighbours[x][0]], neighbours[x][1]) for x in range(len(neighbours))])
                
            elif len(neighbours) > 0:
                G[nodes[intercept]].append((nodes[neighbours[0][0]], neighbours[0][1]))
    
    return G

In [8]:
G_al = createGraph(streets, intercepts, nodes)
G_al[0:10]

[[(5321, 0.001279332564541968),
  (4872, 0.0019002099004250016),
  (2454, 0.0004229534575241186),
  (8573, 0.0009715734739870846)],
 [(1008, 0.002279321782435222),
  (4536, 0.0022857436509651622),
  (1357, 0.0007432058788233657),
  (5223, 0.0007838542254648449)],
 [(3200, 0.0014649646878627464),
  (6538, 0.002909104332341213),
  (4320, 0.0009157862760744407),
  (2191, 0.0025492698662097717)],
 [(5578, 0.0008484919437821345),
  (2376, 0.0009084006418334457),
  (2101, 0.0006936784915827566),
  (869, 0.001718569154048307)],
 [(1085, 0.0007231436583603764),
  (5531, 0.001977372037544103),
  (1828, 0.0008728518115993503),
  (876, 0.0013408286713585027),
  (7151, 0.001293435864896755),
  (1828, 0.0008728518115993503),
  (2074, 0.0021884950339751065)],
 [(7551, 0.00039627977183124603)],
 [(4830, 0.0018692789906829517),
  (8226, 0.001873278978402213),
  (3731, 0.001072010643674649),
  (9221, 0.001074680974494794)],
 [(3659, 0.0007360440921043738),
  (2939, 3.707124060651967e-05),
  (922, 0.000

In [9]:
G_al[G_al[0][0][0]][0]

(5096, 0.0009696380181669667)

## Testing the distance calculation (Km)

In [10]:
nodeToIntercept[0]

(37.7543372313743, -122.40545417189192)

In [11]:
nodeToIntercept[5321]

(37.753063831974, -122.40533110370748)

In [12]:
calculateDistance(nodeToIntercept[0], nodeToIntercept[5321])

0.001279332564541968

In [13]:
assert 0 in [v for v, w in G_al[G_al[0][0][0]]]
assert 0 in [v for v, w in G_al[G_al[0][1][0]]]

# Tests

In [14]:
def commonStreet(ls1, ls2):
    for street in ls1:
        if street in ls2:
            return True
    return False

In [15]:
assert commonStreet(streets[nodeToIntercept[G_al[0][0][0]]], streets[nodeToIntercept[0]])
assert commonStreet(streets[nodeToIntercept[G_al[0][1][0]]], streets[nodeToIntercept[0]])

In [16]:
assert commonStreet(streets[nodeToIntercept[G_al[1][0][0]]], streets[nodeToIntercept[1]]) 
assert commonStreet(streets[nodeToIntercept[G_al[1][1][0]]], streets[nodeToIntercept[1]])

# Writing the graph adjacency list

In [17]:
f = open("weighted_graph.al", "w")
c = 0
for line in G_al:
    f.write(str(c)+' ')
    if len(line) > 0:
        for x, y in line:
            f.write(str(x)+','+str(y)+' ')
    else:
        f.write('-')
    f.write('\n')
    c += 1